In [25]:
!pip install contractions nltk

In [26]:
import pandas as pd
import re
import nltk
import contractions
from nltk.tokenize.toktok import ToktokTokenizer

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
df = pd.read_csv('/content/dataset.csv')
print("Columns in the file:", df.columns)
df.head()

Columns in the file: Index(['Heading', 'Body', 'URL'], dtype='object')


,Heading,Body,URL
0,free speech not hate speech madras high court ...,madras high court issue significant remark ami...,https://www.indiatoday.in/law/high-courts/stor...
1,comment take context say us cop mock indian st...,seattle police officer guild friday come defen...,https://www.indiatoday.in/world/story/indian-s...
2,first meeting one nation one election committe...,first official meeting one nation one election...,https://www.indiatoday.in/india/story/one-nati...
3,us airlines flight depressurize midair plummet...,united airlines jet head rome turn around less...,https://www.indiatoday.in/world/story/us-fligh...
4,terrorist kill security force foil infiltratio...,three terrorist kill infiltration bid foil sec...,https://www.indiatoday.in/india/story/one-terr...


In [29]:
def preprocess(series):
    # convert to lowercase
    series = series.apply(lambda x: str(x).lower())

    # remove contractions
    series = series.apply(lambda x: contractions.fix(x))

    # remove punctuation and special characters
    series = series.str.replace(r'[^\w\s]', '', regex=True)
    series = series.str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

    # remove numbers
    def remove_numbers(text):
        pattern = r'[^a-zA-z.,!?/:;\"\'\s]'
        return re.sub(pattern, '', text)
    series = series.apply(remove_numbers)

    # remove stopwords (but keep 'not')
    tokenizer = ToktokTokenizer()
    stopword_list = nltk.corpus.stopwords.words('english')
    if 'not' in stopword_list:
        stopword_list.remove('not')

    def remove_stopwords(text):
        tokens = tokenizer.tokenize(text)
        tokens = [token.strip() for token in tokens]
        t = [token for token in tokens if token.lower() not in stopword_list]
        text = ' '.join(t)
        return text

    series = series.apply(remove_stopwords)

    return series

In [30]:
df['clean_text'] = preprocess(df['Body'])
df.head()

,Heading,Body,URL,clean_text
0,free speech not hate speech madras high court ...,madras high court issue significant remark ami...,https://www.indiatoday.in/law/high-courts/stor...,madras high court issue significant remark ami...
1,comment take context say us cop mock indian st...,seattle police officer guild friday come defen...,https://www.indiatoday.in/world/story/indian-s...,seattle police officer guild friday come defen...
2,first meeting one nation one election committe...,first official meeting one nation one election...,https://www.indiatoday.in/india/story/one-nati...,first official meeting one nation one election...
3,us airlines flight depressurize midair plummet...,united airlines jet head rome turn around less...,https://www.indiatoday.in/world/story/us-fligh...,united airlines jet head rome turn around less...
4,terrorist kill security force foil infiltratio...,three terrorist kill infiltration bid foil sec...,https://www.indiatoday.in/india/story/one-terr...,three terrorist kill infiltration bid foil sec...


In [31]:
df.to_csv('/content/cleaned_dataset.csv', index=False)
print("✅ Cleaned dataset saved as cleaned_dataset.csv")

✅ Cleaned dataset saved as cleaned_dataset.csv
